In [30]:
import os
import boto3  # AWS
import pandas as pd
from pandas import json_normalize
import numpy as np
import json
import io

# Set the maximum number of rows to 200
pd.set_option("display.max_rows", None) 

# the below extension properly formats a cell after it is run
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Load AWS Files
s3R = boto3.resource("s3")
s3 = boto3.client("s3")
bucket_name = "capstonehaystacks"
# Listing the contents of the S3 bucket to ensure everything looks good
for obj in s3R.Bucket(bucket_name).objects.all():
    print(obj.key)

GA_LISTINGS_SALES.csv
GA_LISTINGS_SALES_V2.csv
GA_listing.csv
all_zips_grocery_store.json
all_zips_restaurant.json
atlanta-geo.csv
atlanta_cbsa_zip.csv
atlanta_hdma_2021.csv
atlanta_hdma_census_2021.csv
atlanta_listings.csv
core_geo_dataset.csv
crime.csv
crime_rating_zipcode.csv
elementary_schools.csv
high_schools.csv
hmda_2021_ga_all-records_labels.csv
listings_with_tract.csv
middle_schools.csv
poi_combined_haystack.csv
points-of-interest-google.csv
points-of-interest-google2.csv
points-of-interest-haystacks.csv
schools.csv


## Housing Data (Base dataframe for merging onto)

In [6]:
file_name = "listings_with_tract.csv"

file_obj = s3.get_object(Bucket=bucket_name, Key=file_name)
listings = pd.read_csv(file_obj["Body"])
print(listings.shape)
listings.head(3)

(9093, 21)


,latitude,longitude,full_street_address,city,county_name,beds,baths_full,baths_half,square_footage,lot_size,year_built,details,special_features,price,transaction_type,listing_status,listing_special_features,census_state_name,census_county_name,zip,census_tract_number
0,33.971863,-83.618736,633 Marigot Way,Statham,Barrow-County,3.0,2.0,0.0,1634.0,1.0,2004.0,"Detached, 3 Beds, 2 Baths, 1,634 Sq Ft",0,305000,1,1,0,Georgia,"Barrow, GA",30666,1.301318e+10
1,34.018024,-83.663994,431 Dunahoo Road,Winder,Barrow-County,3.0,2.0,0.0,1625.0,0.0,1998.0,"Detached, 3 Beds, 2 Baths, 1,625 Sq Ft",0,349900,1,1,0,Georgia,"Barrow, GA",30680,1.301318e+10
2,34.003227,-83.749001,471 Monticello Court,Winder,Barrow-County,4.0,3.0,0.0,1816.0,0.0,2001.0,"Detached, 4 Beds, 3 Baths, 1,816 Sq Ft",0,289900,1,1,0,Georgia,"Barrow, GA",30680,1.301318e+10


In [54]:
# Create core dataset to merge everything into
core = listings.copy()

<IPython.core.display.Javascript object>

## POI Data
- **Groupby** Data by Zip
- **Widen** data by Category(mean_rating, Num_ratings, num POI's)
- **Merge** into listings

In [7]:
file_name = "poi_combined_haystack.csv"

file_obj = s3.get_object(Bucket=bucket_name, Key=file_name)
POI = pd.read_csv(file_obj["Body"])
print(POI.shape)
POI.head(3)

(8473, 11)


,latitude,longitude,state,place_id,name,rating,num_ratings,poi_types,zipcode,address,primary_type
0,33.791987,-84.385677,GA,ChIJoz4--E8E9YgRxpSO8xC1jj0,Residence Inn Atlanta Midtown/Peachtree at 17th,4.1,723,"['lodging', 'point_of_interest', 'establishment']",30309,"1365 Peachtree Street Northeast, Atlanta",lodging
1,33.790216,-84.388776,GA,ChIJh-lI8FoE9YgRyKGKYvLpvA8,Artmore Hotel,3.9,1106,"['lodging', 'point_of_interest', 'establishment']",30309,"1302 West Peachtree Street Northwest, Atlanta",lodging
2,33.799427,-84.392589,GA,ChIJFybLXVQE9YgRTePNqN08VnU,Atlanta Peachtree Station,3.6,163,"['train_station', 'transit_station', 'point_of...",30309,"1688 Peachtree Road Northwest, Atlanta",train_station


In [47]:
# Group POI at zipcode Level

POI_grouped = (
    POI.groupby(["zipcode", "primary_type"])
    .agg({"place_id": "count", "rating": "mean", "num_ratings": "sum"})
    .round({"rating": 2})
)

POI_grouped.columns = ["count", "rating_mean", "num_ratings"]

POI_grouped = POI_grouped.reset_index()

POI_grouped.head(12)

,zipcode,primary_type,count,rating_mean,num_ratings
0,28775,tourist_attraction,1,4.60,247
1,28909,food,1,4.80,369
2,28909,restaurant,1,4.60,86
3,29643,campground,1,4.50,206
4,29821,general_contractor,1,5.00,1
5,29821,point_of_interest,1,4.90,53
6,29842,meal_takeaway,1,4.00,325
7,30002,bar,9,4.47,7641
8,30002,meal_takeaway,3,3.50,1404
9,30002,restaurant,8,4.39,11356


<IPython.core.display.Javascript object>

In [48]:
POI_wide = POI_grouped.pivot_table(index="zipcode", columns="primary_type")

<IPython.core.display.Javascript object>

In [50]:
POI_wide.head(3)

count                                                     \
primary_type accounting airport amusement_park art_gallery atm bakery bank   
zipcode                                                                      
28775               NaN     NaN            NaN         NaN NaN    NaN  NaN   
28909               NaN     NaN            NaN         NaN NaN    NaN  NaN   
29643               NaN     NaN            NaN         NaN NaN    NaN  NaN   

                                                                           \
primary_type bar beauty_salon bicycle_store book_store bowling_alley cafe   
zipcode                                                                     
28775        NaN          NaN           NaN        NaN           NaN  NaN   
28909        NaN          NaN           NaN        NaN           NaN  NaN   
29643        NaN          NaN           NaN        NaN           NaN  NaN   

                                                                            \
primary_type campground car_dealer car_rental car_repair car_wash cemetery   
zipcode                                                                      
28775               NaN        NaN        NaN        NaN      NaN      NaN   
28909               NaN        NaN        NaN        NaN      NaN      NaN   
29643               1.0        NaN        NaN        NaN      NaN      NaN   

                                                                           \
primary_type church city_hall clothing_store convenience_store courthouse   
zipcode                                                                     
28775           NaN       NaN            NaN               NaN        NaN   
28909           NaN       NaN            NaN               NaN        NaN   
29643           NaN       NaN            NaN               NaN        NaN   

                                                                    \
primary_type dentist department_store doctor drugstore electrician   
zipcode                                                              
28775            NaN              NaN    NaN       NaN         NaN   
28909            NaN              NaN    NaN       NaN         NaN   
29643            NaN              NaN    NaN       NaN         NaN   

                                                                               \
primary_type electronics_store finance fire_station florist food funeral_home   
zipcode                                                                         
28775                      NaN     NaN          NaN     NaN  NaN          NaN   
28909                      NaN     NaN          NaN     NaN  1.0          NaN   
29643                      NaN     NaN          NaN     NaN  NaN          NaN   

                                                             \
primary_type furniture_store gas_station general_contractor   
zipcode                                                       
28775                    NaN         NaN                NaN   
28909                    NaN         NaN                NaN   
29643                    NaN         NaN                NaN   

                                                                         \
primary_type grocery_or_supermarket gym hair_care hardware_store health   
zipcode                                                                   
28775                           NaN NaN       NaN            NaN    NaN   
28909                           NaN NaN       NaN            NaN    NaN   
29643                           NaN NaN       NaN            NaN    NaN   

                                                                               \
primary_type home_goods_store hospital insurance_agency jewelry_store laundry   
zipcode                                                                         
28775                     NaN      NaN              NaN           NaN     NaN   
28909                     NaN      NaN              NaN           NaN     NaN   
29643      

<IPython.core.display.Javascript object>

In [52]:
# Combine multi-level column headers into a single level
POI_wide.columns = [
    f"{col[1]}_{col[0]}" if col[1] != "" else col[0] for col in POI_wide.columns
]

<IPython.core.display.Javascript object>

In [53]:
POI_wide.head(3)

,accounting_count,airport_count,amusement_park_count,art_gallery_count,atm_count,bakery_count,bank_count,bar_count,beauty_salon_count,bicycle_store_count,book_store_count,bowling_alley_count,cafe_count,campground_count,car_dealer_count,car_rental_count,car_repair_count,car_wash_count,cemetery_count,church_count,city_hall_count,clothing_store_count,convenience_store_count,courthouse_count,dentist_count,department_store_count,doctor_count,drugstore_count,electrician_count,electronics_store_count,finance_count,fire_station_count,florist_count,food_count,funeral_home_count,furniture_store_count,gas_station_count,general_contractor_count,grocery_or_supermarket_count,gym_count,hair_care_count,hardware_store_count,health_count,home_goods_store_count,hospital_count,insurance_agency_count,jewelry_store_count,laundry_count,lawyer_count,library_count,liquor_store_count,local_government_office_count,lodging_count,meal_delivery_count,meal_takeaway_count,movie_rental_count,movie_theater_count,moving_company_count,museum_count,night_club_count,painter_count,park_count,parking_count,pet_store_count,pharmacy_count,physiotherapist_count,place_of_worship_count,plumber_count,point_of_interest_count,police_count,post_office_count,premise_count,primary_school_count,real_estate_agency_count,restaurant_count,roofing_contractor_count,rv_park_count,school_count,secondary_school_count,shoe_store_count,shopping_mall_count,spa_count,stadium_count,storage_count,store_count,supermarket_count,tourist_attraction_count,train_station_count,transit_station_count,travel_agency_count,university_count,veterinary_care_count,zoo_count,accounting_num_ratings,airport_num_ratings,amusement_park_num_ratings,art_gallery_num_ratings,atm_num_ratings,bakery_num_ratings,bank_num_ratings,...,tourist_attraction_num_ratings,train_station_num_ratings,transit_station_num_ratings,travel_agency_num_ratings,university_num_ratings,veterinary_care_num_ratings,zoo_num_ratings,accounting_rating_mean,airport_rating_mean,amusement_park_rating_mean,art_gallery_rating_mean,atm_rating_mean,bakery_rating_mean,bank_rating_mean,bar_rating_mean,beauty_salon_rating_mean,bicycle_store_rating_mean,book_store_rating_mean,bowling_alley_rating_mean,cafe_rating_mean,campground_rating_mean,car_dealer_rating_mean,car_rental_rating_mean,car_repair_rating_mean,car_wash_rating_mean,cemetery_rating_mean,church_rating_mean,city_hall_rating_mean,clothing_store_rating_mean,convenience_store_rating_mean,courthouse_rating_mean,dentist_rating_mean,department_store_rating_mean,doctor_rating_mean,drugstore_rating_mean,electrician_rating_mean,electronics_store_rating_mean,finance_rating_mean,fire_station_rating_mean,florist_rating_mean,food_rating_mean,funeral_home_rating_mean,furniture_store_rating_mean,gas_station_rating_mean,general_contractor_rating_mean,grocery_or_supermarket_rating_mean,gym_rating_mean,hair_care_rating_mean,hardware_store_rating_mean,health_rating_mean,home_goods_store_rating_mean,hospital_rating_mean,insurance_agency_rating_mean,jewelry_store_rating_mean,laundry_rating_mean,lawyer_rating_mean,library_rating_mean,liquor_store_rating_mean,local_government_office_rating_mean,lodging_rating_mean,meal_delivery_rating_mean,meal_takeaway_rating_mean,movie_rental_rating_mean,movie_theater_rating_mean,moving_company_rating_mean,museum_rating_mean,night_club_rating_mean,painter_rating_mean,park_rating_mean,parking_rating_mean,pet_store_rating_mean,pharmacy_rating_mean,physiotherapist_rating_mean,place_of_worship_rating_mean,plumber_rating_mean,point_of_interest_rating_mean,police_rating_mean,post_office_rating_mean,premise_rating_mean,primary_school_rating_mean,real_estate_agency_rating_mean,restaurant_rating_mean,roofing_contractor_rating_mean,rv_park_rating_mean,school_rating_mean,secondary_school_rating_mean,shoe_store_rating_mean,shopping_mall_rating_mean,spa_rating_mean,stadium_rating_mean,storage_rating_mean,store_rating_mean,supermarket_rating_mean,tourist_attraction_rating_mean,train_station_rati

<IPython.core.display.Javascript object>

#### POI_wide select columns based on correlations

How are we doing the .corr()? It has to be summarized at the same level, so are we doing:
- avg sale price per zipcode compared --> count of POI's per zip 

In [ ]:
# Correllation between POI's and Houses
POI_wide_corr = xxxxx.corr()

In [ ]:
# Selecting columns based on most extreme .corr() vals
POI_wide_sel = POI_wide_corr['','','']

#### Merge POI_wide

In [ ]:
pd.merge(core, POI_wide_sel, how='left', left_on='zip', right_on='zipcode')

## HDMA data

In [61]:
file_name = "atlanta_hdma_2021.csv"
file_obj = s3.get_object(Bucket=bucket_name, Key=file_name)
hdma = pd.read_csv(file_obj["Body"])
print(hdma.shape)
hdma.head(3)

(222, 18)


,zip_code,average_loan_amount_home_improvement_approved,average_loan_amount_home_improvement_denied,average_loan_amount_home_purchase_approved,average_loan_amount_home_purchase_denied,loan_count_home_improvement_approved,loan_count_home_improvement_denied,loan_count_home_purchase_approved,loan_count_home_purchase_denied,total_loan_count,total_approved_loans,total_denied_loans,approval_percentage,zip_median_income,population,zip_owner_occupied_units,total_one_to_four_family_homes,median_age_of_housing_units
0,30002,160100,152390,285173,237129,203,42,624,76,945,827,118,87.51,64488,3129541,549639,936918,44
1,30004,288903,249241,353285,382510,1436,240,4142,339,6157,5578,579,90.60,137621,31727569,8681133,10473381,16
2,30005,294620,233872,304506,335033,862,138,2430,176,3606,3292,314,91.29,166438,16650605,4234171,5065248,15


<IPython.core.display.Javascript object>

#### hdma_select

In [ ]:
hdma_select = hdma['','','','','']

#### Merge hdma into core

In [ ]:
pd.merge(core, hdma_select, how='left', left_on='zip', right_on='zip_code')

## Census Data

In [62]:
file_name = "atlanta_cbsa_zip.csv"
file_obj = s3.get_object(Bucket=bucket_name, Key=file_name)
census = pd.read_csv(file_obj["Body"])
print(census.shape)
census.head(3)

(241, 9)


,Unnamed: 0,census_cbsa_geoid,census_cbsa_name,census_cbsa_lsad,census_cbsa_lat,census_cbsa_lon,census_zcta5_geoid,census_zcta5_lat,census_zcta5_lon
0,3208,12060,"Atlanta-Sandy Springs-Alpharetta, GA",M1,33.693728,-84.399911,30309,33.799851,-84.385837
1,3219,12060,"Atlanta-Sandy Springs-Alpharetta, GA",M1,33.693728,-84.399911,30517,34.130296,-83.797289
2,3222,12060,"Atlanta-Sandy Springs-Alpharetta, GA",M1,33.693728,-84.399911,30548,34.089396,-83.763599


<IPython.core.display.Javascript object>